In [1]:
import numpy as np
import pandas as pd
! pip install mystic

Tenemos la siguiente formulacion

$$
\begin{align*}
min\ \  & 24.55 X_1 + 26.75 X_2 + 39.00 X_3 + 40.50 X_4 \\
s.a\ \  & X_1 + X_2 + X_3 + X_4 = 1 \\
        & 2.3 X_1 +  5.6 X_2 + 11.1 X_3 +  1.3 X_4 \geq  5 \\
        & \mathbb{P}(21 - (\varsigma_1 X_1 + \varsigma_2 X_2 + \varsigma_3 X_3 + \varsigma_4 X_4) \leq 0 ) \geq \alpha \\
        & X_{i} \geq 0.01 \ \ \text{,\{i = 1,2,3,4\}}
\end{align*}
$$

La cual resuelve el problema de minimizar el costo de un conjunto de ingredientes que satisfacen ciertos requerimientos nutricionales, y en los cuales se asume una distribucion normal e independiente entre la cantidad de proteinas de cada ingrediente.

In [2]:
def objetivo(x):
    return (24.55*x[0] + 26.75*x[1] + 39.00*x[2] + 40.50*x[3])

% Funcion quantile 
def percentile(p, x):
    x = np.sort(x)
    p = 0.01 * p * len(x)
    if int(p) != p:
        return x[int(np.floor(p))]
    p = int(p)
    return x[p:p+2].mean()


from mystic.penalty import quadratic_inequality
from mystic.penalty import quadratic_equality

M = 50
x_1 = np.random.normal(12.0, 0.2809, M)
x_2 = np.random.normal(11.9, 0.1936, M)
x_3 = np.random.normal(41.8, 20.250, M)
x_4 = np.random.normal(52.1, 0.6241, M)

varsigma = np.array([x_1,x_2,x_3,x_4]).T



def gen_pen(**kwds):
    def pen1(x, p, v, **kwd): # 95%(xTsx) - v <= 0
        x = np.atleast_2d(x)
        return  (v -  percentile(p, [np.dot(varsigma[i],x.T)[0]-21  for i in range(0,M-1)]))

    def pen2(x, **kwd):
        return (x[0] + x[1] + x[2] + x[3] - 1)
    
    def pen3(x, **kwd):
        return (5 - (2.3* x[0] + 5.6*x[1] + 11.1*x[2] + 1.3*x[3]))
    
    #def pen4(x, **kwd):
    #    return (21 - (12.0* x[0] + 11.9*x[1] + 41.1*x[2] + 52.1*x[3]))
    
    @quadratic_inequality(pen1, k=1e4, kwds=kwds)
    @quadratic_equality( pen2,  k=1e4, kwds=kwds)
    @quadratic_inequality(pen3, k=1e4, kwds=kwds)
    #@quadratic_inequality(pen4, k=1e4, kwds=kwds)
    
    def penalty(x):
        return 0.0
    
    return penalty





parametros_90 = {
    'p': 99,
    'v': 0.9
}
parametros_95 = {
    'p': 99,
    'v': 0.95
}
parametros_99 = {
    'p': 99,
    'v': 0.99
}
from mystic.solvers import diffev2
from mystic.monitors import VerboseMonitor
mon = VerboseMonitor(10)

penalty_90 = gen_pen(**parametros_90)
penalty_95 = gen_pen(**parametros_95)
penalty_99 = gen_pen(**parametros_99)

bounds = [(0.0, None)]*4

result_90 = diffev2(objetivo, x0=bounds,bounds=bounds, penalty=penalty_90, npop=40, gtol=500, disp=False, full_output=True)
result_95 = diffev2(objetivo, x0=bounds,bounds=bounds, penalty=penalty_95, npop=40, gtol=500, disp=False, full_output=True)
result_99 = diffev2(objetivo, x0=bounds,bounds=bounds, penalty=penalty_99, npop=40, gtol=500, disp=False, full_output=True)
#v1,v2,v3,v4 = result[0]

#print(f'El porcentaje de aporte nutricional de cada alimento es: \n Cebada = {v1*100}% \n Avena = {v2*100}% \n Copos de Sésamo = {v3*100}% \n Harina de Maní = {v4*100}%')
#print('El valor óptimo obtenido es: '+ str(result[1]))
#print('El aporte de grasas es: '+str(np.dot(result[0], [2.3 , 5.6 , 11.1 , 1.3])))
#print('El aporte de proteínas es: '+str(np.dot(result[0],[12,11.9, 41.8, 52.1])))
data = {'alpha': [0.9, 0.95, 0.99],
        'X_1':[result_90[0][0],result_95[0][0],result_99[0][0]],
        'X_2':[result_90[0][1],result_95[0][1],result_99[0][1]],
        'X_3':[result_90[0][2],result_95[0][2],result_99[0][2]],
        'X_4':[result_90[0][3],result_95[0][3],result_99[0][3]],
        'Costo':[result_90[1],result_95[1],result_99[1]],
        'Aporte Grasa':[np.dot(result_90[0], [2.3 , 5.6 , 11.1 , 1.3]),np.dot(result_95[0], [2.3 , 5.6 , 11.1 , 1.3]),np.dot(result_99[0], [2.3 , 5.6 , 11.1 , 1.3])],
        'Aporte Proteina':[np.dot(result_90[0],[12,11.9, 41.8, 52.1]),np.dot(result_95[0],[12,11.9, 41.8, 52.1]),np.dot(result_99[0],[12,11.9, 41.8, 52.1])] }

df = pd.DataFrame(data, columns =['alpha', 'X_1','X_2','X_3','X_4','Costo','Aporte Grasa','Aporte Proteina']).set_index('alpha')


UsageError: Line magic function `%` not found.


Resolvemos el problema:

$$
\begin{align*}
min\ \  & -X_2 \\
s.a\ \  & c_2 X_1 + 2 X_2 - u(x,c) + v(x,c)  = 0\\
        & 3 X_1 - u(x,c) - v(x,c) = 0 \\
        & \mathbb{P}(- c_1 + u(x,c) - \frac{1}{2} (c_2+1) X_1 \leq 0) \geq \alpha_1 \\
        & \mathbb{P}( -v(x,c) \leq 0 ) \geq \alpha_2 \\
        & X_{i} \geq 0 \ \ \text{,\{i = 1,2\}}
\end{align*}
$$


In [ ]:
def objetivo2(x):
    return -x[1]

x_11 = np.random.normal(1,0.1,M)
x_22 = np.random.normal(1,0.1,M)

C = np.array([x_11,x_22]).T
C_bar = C.mean(0)

def gen_pen2(**kwds):
    def pen1(x, p, v1, **kwd): # p%(-c1 + x1 + x2) - v <= 0
        #x = np.atleast_2d(x)
        return  (v1 +  percentile(p, [(-C[i][0] + x[0] + x[1]) for i in range(0,M-1)]) - 1)

    def pen2(x,p,v2, **kwd):
        #x = np.atleast_2d(x)
        return (v2 - percentile(p, [(3 - C[i][1] - x[1]) for i in range(0,M-1)]))
    
    
    @quadratic_inequality(pen1, k=1e12, kwds=kwds)
    @quadratic_inequality(pen2, k=1e12, kwds=kwds)
    def penalty(x):
        return 0.0
#     
    return penalty

parametros2 = {
    'p': 99,
    'v1': 0.9987,
    'v2': 0.9987
}
penalty2 = gen_pen2(**parametros2)

bounds2 = [(0,1)]*2

result = diffev2(objetivo2, x0=[0.5,0.5],bounds=bounds2, penalty=penalty2, npop=40, gtol=500, disp=False, full_output=True)

print(result[0])
print(result[1])

In [ ]:
def gen_prueba():
    def pen11(x): ## <= 0
        return x[0] + x[1] - C_bar[0]
    def pen22(x):  ## <= 0
        return - 0.5*(3-C_bar[1])*x[0] + x[1]
    @quadratic_inequality(pen11, k=1e4)
    @quadratic_inequality(pen22, k=1e4)
    def penalty(x):
        return 0.0
    return penalty

penalty_prueba = gen_prueba()
result1 = diffev2(objetivo2, x0=bounds2,bounds=bounds2, penalty=penalty_prueba, npop=40, gtol=500, disp=False, full_output=True)
print(result1[0])
print(result1[1])